# Electrode Localization

Change into the parent directory to find the `dj_local_conf.json` file.

In [1]:
import os
import datajoint as dj
# change to the upper level folder to detect dj_local_conf.json
if os.path.basename(os.getcwd())=='notebooks': os.chdir('..')
assert os.path.basename(os.getcwd())=='workflow-array-ephys', ("Please move to the "
                                                               + "workflow directory")

## Coordinate Framework

The Allen Institute hosts [brain atlases](http://download.alleninstitute.org/informatics-archive/current-release/mouse_ccf/annotation/ccf_2017) and [ontology trees](https://community.brain-map.org/t/allen-mouse-ccf-accessing-and-using-related-data-and-tools/359) that we'll use in the next section. The `localization.py` script assumes this is your first atlas, and that you'll use the 100μm resolution. For finer resolutions, edit `voxel_resolution` in `localization.py`. Higher resolution `nrrd` files are quite large when loaded. Depending on the python environment, the terminal may be killed when loading so much information into memory. To load multiple atlases, increment `ccf_id` for each unique atlas.

To run this pipeline ...
1. Download the 100μm `nrrd` and `csv` files from the links above.
2. Move these files to your ephys root directory.

Next, we'll populate the coordinate framework schema simply by loading it.

In [ ]:
from workflow_array_ephys.localization import coordinate_framework as ccf

Now, to explore the data we just loaded.

In [9]:
ccf.BrainRegionAnnotation.BrainRegion()

"ccf_id CCF ID, a.k.a atlas ID",acronym CHARACTER SET utf8 COLLATE utf8_bin,region_name,region_id,color_code hexcode of the color code of this region
0,6b,Layer 6b isocortex,16,8ADA87
0,A13,Dopaminergic A13 group,796,F2483B
0,AAA,Anterior amygdalar area,23,80C0E2
0,ACA,Anterior cingulate area,31,40A666
0,ACA1,Anterior cingulate area layer 1,572,40A666
0,ACA2/3,Anterior cingulate area layer 2/3,1053,40A666
0,ACA5,Anterior cingulate area layer 5,739,40A666
0,ACA6a,Anterior cingulate area layer 6a,179,40A666
0,ACA6b,Anterior cingulate area layer 6b,227,40A666
0,ACAd,Anterior cingulate area dorsal part,39,40A666


Let's look at the dimensions of the primary somatosensory cortex, which has the acronym `SSp1`. To look at other regions, open the CSV you downloaded and search for your desired region.

In [11]:
ccf.BrainRegionAnnotation.Voxel() & 'acronym="SSp1"'

"ccf_id CCF ID, a.k.a atlas ID",acronym CHARACTER SET utf8 COLLATE utf8_bin,x (um) Anterior-to-Posterior (AP axis),y (um) Superior-to-Inferior (DV axis),z (um) Left-to-Right (ML axis)
0,SSp1,5700,5400,5400
0,SSp1,5700,5400,5500
0,SSp1,5700,5400,5600
0,SSp1,5700,5400,5800
0,SSp1,5700,5400,5900
0,SSp1,5700,5500,5500
0,SSp1,5700,5500,5600
0,SSp1,5700,5500,5800
0,SSp1,5700,5500,5900
0,SSp1,5700,5600,5600


## Electrode Localization

If you have `channel_location` json files for your data, you can look at the position and regions associated with each electrode. Here, we've added an example file to our pre-existing `subject6` for demonstration purposes.

In [6]:
dj.config.load('dj_local_conf.json')
from workflow_array_ephys.pipeline import subject, session, ephys, probe
from workflow_array_ephys.localization import electrode_localization as eloc

In [7]:
from workflow_array_ephys.localization import electrode_localization as eloc
from workflow_array_ephys.localization import coordinate_framework as ccf

In [ ]:
eloc.ElectrodePosition.populate()
eloc.ElectrodePosition.Electrode()

In [2]:
dj.conn()

Connecting cbroz@tutorial-db.datajoint.io:3306


DataJoint connection (connected) cbroz@tutorial-db.datajoint.io:3306

In [4]:
mydiagram=dj.Diagram(subject).make_dot()
mydiagram.write(path='mydiagram_dotfile',format='dot')

801

In [25]:
((dj.Di(ccf) + dj.Di(eloc) -1) + ephys.CuratedClustering + ephys.CuratedClustering.Unit 
 + ephys.EphysRecording
 + ephys.Clustering).make_dot().write(path='temp_eloc_full', format='dot')

True

In [6]:
from workflow_array_ephys import analysis 
from workflow_array_ephys.pipeline import trial, event
from element_trial.event import *
from element_trial.trial import *
from element_session.session_with_datetime import Session
from element_array_ephys.ephys_no_curation import *

In [7]:
#dj.Di(trial) + dj.Di(event) -1 + 
(dj.Di(analysis) - 1 + BehaviorRecording + Session)#.make_dot().write(path='temp_ev', format='dot')

DataJointError: Class CuratedClustering is not properly declared (schema decorator not applied?)

In [32]:
from workflow_array_ephys import analysis

In [31]:
dj.schema('cbroz_analysis').list_tables()

['spikes_alignment_condition',
 'spikes_alignment_condition__trial',
 '__spikes_alignment',
 '__spikes_alignment__aligned_trial_spikes',
 '__spikes_alignment__unit_p_s_t_h']